In [124]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv


In [150]:
test=pd.read_csv('/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv')
train=pd.read_csv('/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv')
if 'Unnamed: 0' in train.columns:
    train = train.drop(columns=['Unnamed: 0'])
if 'Unnamed: 0' in test.columns:
    test = test.drop(columns=['Unnamed: 0'])
train.head()
#Check total of NaN vals in cols 
print(train.isnull().sum())
numeric_cols=train.select_dtypes(include=['int64','float64']).columns
train[numeric_cols]=train[numeric_cols].fillna(train[numeric_cols].median())
train.head

ID               0
class            0
20150720_N     560
20150602_N    1200
20150517_N     800
20150501_N     960
20150415_N     480
20150330_N    1120
20150314_N     720
20150226_N    1360
20150210_N     640
20150125_N    1040
20150109_N     880
20141117_N    1280
20141101_N     400
20141016_N    1440
20140930_N     800
20140813_N     560
20140626_N    1600
20140610_N     480
20140525_N     720
20140509_N     880
20140423_N    1760
20140407_N     640
20140322_N    1120
20140218_N    1440
20140202_N     560
20140117_N    1200
20140101_N     400
dtype: int64


<bound method NDFrame.head of          ID       class  20150720_N  20150602_N  20150517_N  20150501_N  \
0         1       water    637.5950     658.668   -1882.030   -1924.360   
1         2       water    634.2400     593.705   -1625.790   -1672.320   
2         4       water     58.0174   -1599.160    4672.000   -1052.630   
3         5       water     72.5180    5513.375     380.436   -1256.930   
4         8       water   1136.4400    5513.375    4672.000    1647.830   
...     ...         ...         ...         ...         ...         ...   
7995  10538  impervious   1207.7000     984.620    4672.000    1166.250   
7996  10539  impervious   2170.3500    1419.720    1361.000    1478.710   
7997  10542  impervious   1895.6800    1454.740    4672.000    1033.560   
7998  10543  impervious   3465.7400    1283.320     413.412    4391.050   
7999  10545  impervious   6941.1900    1667.870    5084.780    5810.755   

      20150415_N  20150330_N  20150314_N  20150226_N  ...  20140610_N

In [151]:
X_train=train.drop(columns=['ID','class'])
y_train=train['class']
X_test=test.drop(columns=['ID'])
#check outliers
for col in X_train.columns:
    Q1=X_train[col].quantile(0.25)
    Q3=X_train[col].quantile(0.75)
    IQR=Q3-Q1
    lower=Q1-1.5*IQR
    upper=Q3+1.5*IQR
    X_train[col] = X_train[col].clip(lower, upper)
    outliers=X_train[(X_train[col]<lower) | (X_train[col]>upper)]
    print(f"{col}: {len(outliers)} outliers")

20150720_N: 0 outliers
20150602_N: 0 outliers
20150517_N: 0 outliers
20150501_N: 0 outliers
20150415_N: 0 outliers
20150330_N: 0 outliers
20150314_N: 0 outliers
20150226_N: 0 outliers
20150210_N: 0 outliers
20150125_N: 0 outliers
20150109_N: 0 outliers
20141117_N: 0 outliers
20141101_N: 0 outliers
20141016_N: 0 outliers
20140930_N: 0 outliers
20140813_N: 0 outliers
20140626_N: 0 outliers
20140610_N: 0 outliers
20140525_N: 0 outliers
20140509_N: 0 outliers
20140423_N: 0 outliers
20140407_N: 0 outliers
20140322_N: 0 outliers
20140218_N: 0 outliers
20140202_N: 0 outliers
20140117_N: 0 outliers
20140101_N: 0 outliers


In [152]:
imputer = SimpleImputer(strategy='median')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Step 2: Scale the imputed data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)
X_valid_scaled = scaler.transform(X_valid)


# Step 3: Train the model
model = LogisticRegression(max_iter=5000)
model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=5000)

In [146]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
y_valid_pred = model.predict(X_valid_scaled)
accuracy = accuracy_score(y_valid, y_valid_pred)
print("Validation Accuracy:", accuracy)

Validation Accuracy: 0.879375


In [155]:
y_test_pred = model.predict(X_test_scaled)
submission = pd.DataFrame({
    'ID': test['ID'],
    'class': y_test_pred
})
submission.to_csv('submission.csv', index=False)
print(submission.head())

   ID    class
0   1     farm
1   2   forest
2   3  orchard
3   4   forest
4   5   forest
